In [4]:
def utm_finder(raster):
    """
    Find the UTM EPSG code of a non-utm projected raster.
    Arguments:  
    raster: input raster path
    Returns:
    UTM EPSG code of the input raster
    """
    with rasterio.open(raster) as dataset:       
        bbox  = dataset.bounds
        bbox_wgs84 = rasterio.warp.transform_bounds(dataset.crs,'EPSG:4326', bbox[0],bbox[1],bbox[2],bbox[3])

        utm_crs_list = query_utm_crs_info(     
            datum_name='WGS 84',
            area_of_interest=AreaOfInterest(
            west_lon_degree=bbox_wgs84[0],
            south_lat_degree=bbox_wgs84[1],
            east_lon_degree=bbox_wgs84[2],
            north_lat_degree=bbox_wgs84[3],),) 

        utm_crs = '{}:{}'.format(utm_crs_list[0].auth_name,utm_crs_list[0].code) 
    return utm_crs

In [5]:
def resample(raster,pixel_size,output_dir,output_name):
    """
    Resample a raster to a given pixel size.
    
    Arguments:  
        raster: input raster path
        pixel_size: resolution in meters
        output_dir: output folder
        output_name: raster name with extension
    
    Returns:
        Resampled raster   
    """
    with rasterio.open(raster) as dataset:
        kwargs = dataset.meta.copy()
        # resample data to target shape
        upscale_factor = dataset.transform[0]/pixel_size
        
        data = dataset.read(out_shape=(dataset.count,int(dataset.height * upscale_factor),int(dataset.width * upscale_factor)),resampling=Resampling.bilinear)
        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2]))
        
        # writing file to disk 
        kwargs.update({'transform': transform,
                       'width': data.shape[2],
                       'height': data.shape[1]})
        
        with rasterio.open(os.path.join(output_dir,output_name), 'w', **kwargs) as dst:
            dst.write(data[0], 1)

In [6]:
def calibration(raster,cosys,output_dir,output_name):
    """
    Raster calibration,speckle reduction, and reprojection.
    
    Arguments:  
        raster: input raster path
        cosys: epsg code of the desired coordinate system
        output_dir: output folder
        output_name: raster name with extension
    
    Returns:
        Calibrated raster in desired projection     
    """
    with rasterio.open(raster) as dataset:
        # tranformation calculation for UTM reprojection
        transform, width, height = calculate_default_transform(
            dataset.crs,
            cosys,
            dataset.width,
            dataset.height,
            *dataset.bounds)
        
        kwargs = dataset.meta.copy()
        kwargs.update({
            'crs': cosys,
            'transform': transform,
            'width': width,
            'height': height,
            'compress': 'lzw',
            'dtype':'float32',
            'nodata':np.nan})
        
        # Calibration
        band = dataset.read(1)
        band = np.float32(np.where(band == 0, np.nan, band))
        band_calib = 20*np.log10(band)-83

        # Filtering
        med =  medfilt2d(band_calib, kernel_size=3)
                  
        with rasterio.open(os.path.join(output_dir,output_name), "w", **kwargs) as dst:           
            for i in range(1, dataset.count + 1):
                reproject(
                    source=med,
                    destination=rasterio.band(dst, i),
                    src_transform=dataset.transform,
                    src_crs=dataset.crs,
                    dst_transform=transform,
                    dst_crs=cosys,
                    resampling=Resampling.nearest)
            
        #memory reduction
        med = None
        band = None
        band_calib = None
                     